# Coffee Market Analysis
## Data-Wrangling Notebook

### Matthew Garton - February 2019

**Purpose:** The purpose of this notebook is to acquire my data, inspect it, clean it and prepare it for EDA and modeling.

**Context**: The ultimate goal of my project is to develop trading signals for coffee futures. I will attempt to build a machine learning model which uses fundamental and technical data to predict the future direction of coffee futures price changes. My expectation at the outset of this project is that my feature matrix will include data on weather, GDP, and coffee production and exports in major coffee-producing nations, GDP and coffee import data in major coffee-importing nations, as well as volume, open-interest, and commitment of traders data for ICE coffee futures contracts.

Note that many of the decisions made and functions written here came up at various stages of the project, from initial inspection all the way to model-building (as is the non-linear nature of the data science workflow). To keep things clean, I have moved all of the data cleaning/prep (outside of train-test splitting and some feature engineering) to this notebook. The csv file that I output can then be accessed in other notebooks in this repository.

In [1]:
import pandas as pd
import datetime

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

## Gathering data

1. Price data (1973-2019) - daily OHLC prices (plus Volume and OI) for ICE Coffee 'C' futures.

    source: [Wiki Continuous Futures database on Quandl](https://www.quandl.com/data/CHRIS-Wiki-Continuous-Futures)
      
      
2. Weather data (1991-2015) - monthly average temperature (celsius) and rainfall (mm) for the top five coffee exporting countries (Brazil, Vietnam, Colombia, Indonesia, Ethiopia).
    
    source: [World Bank Climate Change Knowledge Portal](http://sdwebx.worldbank.org/climateportal/index.cfm?page=downscaled_data_download&menu=historical)
    
    
3. Fundamental data (1990-2017) - annual data on coffee production, imports, exports, etc. from International Coffee Organization*.

    source: [International Coffee Organization](http://www.ico.org/new_historical.asp?section=Statistics)


4. Positioning data (1995-2016) - monthly Commitment of Traders' reports from CFTC

    source: [Commodity Futures Trading Commission](https://www.cftc.gov/MarketReports/CommitmentsofTraders/HistoricalCompressed/index.htm)
    
*Note: Before getting started here, I did some initial data assembling/cleaning in excel, so if you choose to get the data directly from the sources listed above, some preparation will be necessary before getting it into the format shown here. The biggest decision I made so far was in how to handle some of the ICO data which was indexed by 'Crop Year' rather than 'Calendar Year'. My initial solution is to treat the most recent year of the 'Crop Year' as the relevant 'year' for the data (so Crop Year 1991/1992 is treated as Year 1992, with the understanding that all of the data for the 1991-1992 period would have been availably by EOY 1992). For now, this is a simplifying assumption to avoid any 'look-ahead bias.' This might be an oversimplification that I'll have to come back to. 

In [2]:
# import Daily ICE Coffee 'C' Futures price data
coffee = pd.read_csv('../data/CHRIS-ICE_KC1.csv')

# import Monthly Weather data for major coffee producing countries
weather = pd.read_csv('../data/Weather.csv')

# import Annual fundamental (Production, Exports, Imports, etc.) data
fundamental = pd.read_csv('../data/SupplyDemand.csv')

# import Monthly Commitment of Traders report data
cot = pd.read_csv('../data/CommitmentOfTraders.csv')

## Data Preparation

While some of the initial data cleaning took place in Excel before importing, there are a few things I need to handle to get these four dataset into one dataframe that will be ready for visualization, EDA, and then feature engineering.

In particular, the weather data needs to be reshaped. Because I downloaded separate weather data for each country, the weather data includes separate rows for each Date-Country observation. But I want my data to be indexed by Date for modeling, with the specific country data to represent features of that observation, rather than separate observations themselves. To accomplish this, I use a very simple version of the 'Split-Apply-Combine' paradigm. I separate each country's weather data into it's own smaller dataframe, rename the relevant coluns to be tagged with country name, then combine them back into one dataframe indexed on Date.

In [ ]:
# Quick fix to 'Country' column typo..
weather.rename(index=str, columns={' Country':'Country'}, inplace=True)

In [3]:
coffee.head()

,Date,Open,High,Low,Settle,Change,Wave,Volume,Prev. Day Open Interest,EFP Volume,EFS Volume,Block Volume
0,2019-02-11,102.60,102.70,99.85,100.20,-2.40,101.01,41306.0,81909.0,733.0,1851.0,NaN
1,2019-02-08,104.25,104.75,102.25,102.60,-1.70,103.24,39198.0,91190.0,384.0,2525.0,NaN
2,2019-02-07,105.20,105.30,103.55,104.30,-1.20,104.41,38973.0,103661.0,385.0,119.0,NaN
3,2019-02-06,104.70,105.90,104.35,105.50,0.65,105.09,23725.0,106848.0,483.0,18.0,NaN
4,2019-02-05,105.80,106.20,104.25,104.85,-0.75,105.16,21214.0,110696.0,268.0,15.0,NaN


In [5]:
coffee['Open'] = coffee['Open']/100

In [6]:
coffee.head()

,Date,Open,High,Low,Settle,Change,Wave,Volume,Prev. Day Open Interest,EFP Volume,EFS Volume,Block Volume
0,2019-02-11,1.0260,102.70,99.85,100.20,-2.40,101.01,41306.0,81909.0,733.0,1851.0,NaN
1,2019-02-08,1.0425,104.75,102.25,102.60,-1.70,103.24,39198.0,91190.0,384.0,2525.0,NaN
2,2019-02-07,1.0520,105.30,103.55,104.30,-1.20,104.41,38973.0,103661.0,385.0,119.0,NaN
3,2019-02-06,1.0470,105.90,104.35,105.50,0.65,105.09,23725.0,106848.0,483.0,18.0,NaN
4,2019-02-05,1.0580,106.20,104.25,104.85,-0.75,105.16,21214.0,110696.0,268.0,15.0,NaN


In [ ]:
# For each dataframe, index by Date (as datetime object) and extract year, month
dfs = [coffee, weather, fundamental, cot]
for df in dfs:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
# coffee and cot are sorted backwards; reverse order
coffee.sort_index(inplace=True)
cot.sort_index(inplace=True);

In [ ]:
# For weather data, what I want is one row per observation, with each country's
# data represented in columns of that row

countries = ['BRA', 'COL', 'ETH', 'IDN', 'VNM']

# split weather into dfs for each country and rename columns appropriately
dfs = []
for country in countries:
    df = weather[weather['Country'] == country]
    df.rename(index=str, 
              columns={'Temperature (Monthly – C)':'{}_Temp'.format(country),
                       'Precip (mm)':'{}_Precip'.format(country)}, inplace=True)
    df.drop(columns=['Country'], inplace=True)
    dfs.append(df)

# combine separate countries' weather data into one frame indexed by date
weather = dfs[0]

for df in dfs[1:]:
    cols = df.columns.difference(weather.columns)
    weather = weather.merge(df[cols], left_index=True, right_index=True, how='outer')

In [ ]:
weather.index = pd.to_datetime(weather.index)

## Putting it all together

Finally, I combine the four individual dataframes into one master dataframe which feeds into the next stage of my workflow - EDA. There is a slight problem with the periodicity of the data, in that my explanatory variables are not as granular as what I'm trying to predict - prices. This is not ideal and also presents a technical problem when thinking about creating a feature matrix (i.e. lots of NaN values). From a modeling perspective, I may need to circle back to try and get more granular data (daily weather data, for example). However, this level of granularity could be unnecessary for my purposes. I will handle the practical aspect of filling in NaN values in the next stage, but for now I will maintain the integrity of the data (i.e. only export the data that I have). This will make it easier to test out different methods of cleaning up the data in future stages, without being locked into a particular decision now.

In [ ]:
# combine all data into one dataframe
dfs = [coffee, weather, fundamental, cot]

full_data = pd.concat(dfs, axis=1)

In [ ]:
full_data.columns

In [ ]:
# Sample of data where datasets overlap (~1995-2015)
coffee_data = full_data['1994':'2016']

# Drop unnecessary columns
cols_to_drop = ['Change', 'Wave', 'Prev. Day Open Interest',
                'EFP Volume', 'EFS Volume', 'Block Volume',
                'Year']

coffee_data.drop(columns=cols_to_drop, inplace=True);

In [ ]:
# Export to csv
coffee_data.to_csv('../data/CoffeeDataset.csv')